In [1]:
from typing import List, Tuple, Callable, Optional
import numpy as np
from scipy.optimize import shgo
import plotly.graph_objects as go
from scipy.spatial.distance import cdist

In [2]:
def plot_heatmap(
    func, func_name, X_bounds, interest_region,
    points_data: Optional[Tuple[np.ndarray, np.ndarray, str]] = None,
    outliers_data: Optional[Tuple[np.ndarray, np.ndarray, str]] = None
):
    # Create a grid of points
    xx, yy = np.meshgrid(
        np.arange(X_bounds[0, 0], X_bounds[0, 1], 0.01),
        np.arange(X_bounds[1, 0], X_bounds[1, 1], 0.01),
    )
    grid_points = np.c_[xx.ravel(), yy.ravel()]

    # Set Colorbar to use for plots
    colorbar = dict(
        len=0.8,
        thickness=30,
        title_text='Target',
        title_side='bottom',
        y=0., yanchor='bottom',  # Anchor point for vertical positioning
        x=1.0, xanchor='left',  # Anchor point for horizontal positioning
    )

    # Loss function values
    Z = func(grid_points, interest_region)
    Z = Z.reshape(xx.shape)

    # Create the heatmap
    # Create the heatmap with countours
    func_heatmap = go.Heatmap(
        x=xx[0], 
        y=yy[:, 0],
        z=Z,
        colorscale='viridis',
        colorbar=colorbar,
    )

    interest_boundary = go.Contour(
        x=xx[0],
        y=yy[:, 0],
        z=Z,
        contours=dict(
            start=interest_region[0, 0],
            end=interest_region[0, 1],
            size=np.diff(interest_region, axis=1).item(),
            coloring='none',
            showlabels=True,
            labelfont=dict(size=8, color='white')
        ),
        showscale=False,
        line=dict(width=2, color='red', dash='dash'),
        hoverinfo='skip',
        name=f"Interest Boundaries",
        showlegend=True
    )

    data = [func_heatmap, interest_boundary]

    # Add points trace if points_data is provided
    if points_data is not None:
        X_points, y_points, points_label = points_data
        X_points = np.atleast_2d(X_points)
        points_trace = go.Scatter(
            x=X_points[:, 0],
            y=X_points[:, 1],
            name=points_label,
            mode='markers',
            marker=dict(
                color='red',
                size=7,
                symbol='circle',
                line_width=1
            ),
            text=[
                f'X1: {x1:.4f}<br>'
                f'X2: {x2:.4f}<br>'
                f'y : {y:.4f}'
                for (x1, x2), y in zip(X_points, y_points)
            ],
            hoverinfo='text'
        )
        data.append(points_trace)

    # Add outliers trace if outliers_data is provided
    if outliers_data is not None:
        X_outliers, y_outliers, outliers_label = outliers_data
        X_outliers = np.atleast_2d(X_outliers)
        outliers_trace = go.Scatter(
            x=X_outliers[:, 0],
            y=X_outliers[:, 1],
            name=outliers_label,
            mode='markers',
            marker=dict(
                color='black',
                size=7,
                symbol='x-thin-open',
                line_width=2
            ),
            text=[
                f'X1: {x1:.4f}<br>'
                f'X2: {x2:.4f}<br>'
                f'y : {y:.4f}'
                for (x1, x2), y in zip(X_outliers, y_outliers)
            ],
            hoverinfo='text'
        )
        data.append(outliers_trace)

    fig_size = 700

    fig = go.Figure(
        data=data,
        layout=dict(
            xaxis_title="X1",
            yaxis_title="X2",
            showlegend=True,
            width=fig_size,
            height=fig_size,  # Keep the overall figure square aspect_ratio = 1
            xaxis=dict(
                range=X_bounds[0, :],
                constrain='domain',  # Ensures x-axis stays within its domain
            ),
            yaxis=dict(
                range=X_bounds[1, :],
                constrain='domain',  # Ensures y-axis stays within its domain
                scaleanchor="x",
                scaleratio=1
            ),  # Keep the same scale for x and y axes
            title=f"{func_name} Function Heatmap with Interest Region [{interest_region[0, 0]}, {interest_region[0, 1]}]",
        )
    )

    fig.show()

### Function to Map Values from [0, 1] to Interest Region Within a Target Space

In [3]:
def linear_interest_mapping(
    y_raw: np.ndarray,
    interest_bounds: List[Tuple[float, float]], 
    y_raw_center: float = 0.5,
    y_raw_bandwidth: float = 0.15
) -> np.ndarray:
    """
    Applies a piecewise linear mapping to raw values based on interest bounds.

    This function maps the input values as follows:
    - y_raw = 0            -> y = 0
    - y_raw = y_raw_lower  -> y = lower bound of interest region
    - y_raw = y_raw_center -> y = center of interest region
    - y_raw = y_raw_upper  -> y = upper bound of interest region
    - y_raw = 1            -> y = 1

    Args:
        y_raw (np.ndarray): Input array of shape (n_samples,) in range [0, 1].
        interest_bounds (List[Tuple[float, float]]): Min and max interest values for each output dimension.
        y_raw_center (float): Center of the interest region in y_raw space. Default is 0.5.
        y_raw_bandwidth (float): Width of the interest region in y_raw space. Default is 0.1.

    Returns:
        np.ndarray: Mapped output array y of shape (n_samples, n_dim_y) in range [0, 1].
    """
    # Compute lower and upper bounds in y_raw space
    y_raw_lower = y_raw_center - y_raw_bandwidth / 2
    y_raw_upper = y_raw_center + y_raw_bandwidth / 2

    y_raw = np.atleast_1d(y_raw)  # Ensure y_raw is at least 1D
    n_samples = y_raw.shape[0]
    n_dim_y = len(interest_bounds)  # Get dimension of target space

    # Create masks for each piece of the piecewise function
    mask_lower = y_raw <= y_raw_lower
    mask_lower_center = (y_raw > y_raw_lower) & (y_raw <= y_raw_center)
    mask_center_upper = (y_raw > y_raw_center) & (y_raw <= y_raw_upper)
    mask_upper = y_raw > y_raw_upper

    # Calculate bounds and center of interest region
    interest_bounds = np.array(interest_bounds)
    interest_lower = interest_bounds[:, 0]
    interest_upper = interest_bounds[:, 1]
    interest_centers = np.mean(interest_bounds, axis=1)

    # Initialize output array
    y = np.zeros((n_samples, n_dim_y))

    # Piecewise linear mapping
    for i in range(n_dim_y):
        # 0 to lower bound
        y[mask_lower, i] = (interest_lower[i] / y_raw_lower) * y_raw[mask_lower]

        # Lower bound to center
        y[mask_lower_center, i] = interest_lower[i] + (interest_centers[i] - interest_lower[i]) * (y_raw[mask_lower_center] - y_raw_lower) / (y_raw_center - y_raw_lower)

        # Center to upper bound
        y[mask_center_upper, i] = interest_centers[i] + (interest_upper[i] - interest_centers[i]) * (y_raw[mask_center_upper] - y_raw_center) / (y_raw_upper - y_raw_center)

        # Upper bound to 1
        y[mask_upper, i] = interest_upper[i] + (1 - interest_upper[i]) * (y_raw[mask_upper] - y_raw_upper) / (1 - y_raw_upper)

    return y

# Spherical Shell Interest Function

In [4]:
def spherical_shell_interest_function(X: np.ndarray, interest_bounds: List[Tuple[float, float]]) -> np.ndarray:
    """
    Maps design space points X to interest values y based on their distance from the center.

    The function creates a spherical shell of interest in the design space:
    - Center of hypercube (X_dist = 0): y = [0]*n_dim_y
    - Mid-distance points (X_dist = 0.5): y = center of interest region
    - Corners of hypercube (X_dist = 1): y = [1]*n_dim_y

    Args:
        X (np.ndarray): Input array of shape (n_samples, n_dim_X) in range [0, 1].
        interest_bounds (List[Tuple[float, float]]): Min and max interest values for each output dimension.

    Returns:
        np.ndarray: Interest values y of shape (n_samples, n_dim_y).
    """
    X = np.atleast_2d(X)
    n_dim_X = X.shape[1]

    # Define the center of the hypercube
    hypercube_center = np.full(X.shape, 0.5)

    # Calculate maximal distance from center (half of hypercube diagonal)
    max_dist = np.sqrt(n_dim_X) / 2

    # Compute normalized distances from center (0 at center, 1 at corners)
    X_dist = np.linalg.norm(X - hypercube_center, axis=1).ravel() / max_dist

    # Compute interest values using piecewise linear function
    # Inner shell: linear increase from 0 to interest_center
    # Outer shell: linear increase from interest_centers to 1
    y = linear_interest_mapping(X_dist, interest_bounds)

    return y

In [5]:
# Set input space
n_dim_X = 2
X_bounds = np.array([(0, 1)]*n_dim_X)

# Define interest region
interest_region = np.array([[0.7, 0.8]])

# Plot heatmap
plot_heatmap(spherical_shell_interest_function, 'Spherical Shell Interest', X_bounds, interest_region)

# Shubert-like Interest Function

In [6]:
def shubert_like_interest_function(X: np.ndarray, interest_bounds: List[Tuple[float, float]]) -> np.ndarray:
    """
    A multi-dimensional function based on the Shubert-like equation with linear interest region mapping.
    This function has multiple local minima and returns an output with n_dim_y dimensions.
    It linearly maps the raw output to:
    - y = 0 when y_raw = 0
    - y = center of interest region when y_raw = 0.5
    - y = 1 when y_raw = 1

    Args:
        X (np.ndarray): Input array of shape (n_samples, n_dim_X) in range [0, 1].
        interest_bounds (List[Tuple[float, float]]): Min and max interest values for each output dimension.

    Returns:
        np.ndarray: Output array y of shape (n_samples, n_dim_y) in range [0, 1].
    """
    X = np.atleast_2d(X)
    n_samples, n_dim_X = X.shape
    
    # Transform [0, 1]^n_dim_X into [-1, 1]^n_dim_X
    X_transformed = 2 * (X - 0.5)

    # Angular frequency (omega)
    omega = 10 * np.pi

    # Compute Shubert-like function for each input dimension
    shubert_terms = 0.5 * ((1 - np.abs(X_transformed)) * np.cos(omega * X_transformed**2) + 1)
    y_raw = np.mean(shubert_terms, axis=1)

    # Apply linear mapping to interest region
    y = linear_interest_mapping(y_raw, interest_bounds)

    return y

In [7]:
# Set input space
n_dim_X = 2
X_bounds = np.array([(0, 1)]*n_dim_X)

# Define interest region
interest_region = np.array([[0.7, 0.8]])

# Plot heatmap
plot_heatmap(shubert_like_interest_function, 'Shubert-like Interest', X_bounds, interest_region)

# A proxy fast simulator with artificial outlier regions

In [8]:
class FastSimulator:
    def __init__(
        self,
        proxy_function: Callable[[np.ndarray, List[Tuple[float, float]]], np.ndarray],
        n_dim_X: int,
        interest_region: List[Tuple[float, float]],
        n_outlier_regions: int = 1,
        outlier_radius: float = 0.05
    ):
        self._proxy_function = proxy_function
        self.interest_region = np.array(interest_region)
        self.n_dim_X = n_dim_X
        self.n_dim_y = len(interest_region)
        self.outlier_radius = outlier_radius
        self.X_interest = self.find_interest_sample()
        self.X_outliers = self.set_outlier_regions_centers(n_outlier_regions)
        self.remove_close_outliers()

    def proxy_function(self, X: np.ndarray) -> np.ndarray:
        return self._proxy_function(X, self.interest_region)

    def find_interest_sample(self) -> np.ndarray:
        """
        Search for an interest sample where y is in the interest_region using
        SHGO algorithm.
        """
        print(f"{self.__class__.__name__} - > Searching for interest sample...")

        # Define objective function for SHGO
        lower_bounds, upper_bounds = self.interest_region.T

        def objective(x: np.ndarray) -> float:
            """
            Objective function for SHGO to minimize.
            
            Calculates the squared deviation from the interest region.
            Returns 0 if y is within the region, positive penalty otherwise.
            numpy.maximum compares two arrays and return a new array containing the
            element-wise maxima, like np.max([...], axis=1).
            """
            y = self.proxy_function(x.reshape(1, -1)).ravel()
            penalties = np.maximum(lower_bounds - y, 0)**2 + np.maximum(y - upper_bounds, 0)**2
            return np.sum(penalties)

        # Define bounds for each input dimension, avoiding the origin outlier region
        bounds = [(self.outlier_radius, 1)] * self.n_dim_X

        # Run SHGO optimization
        result = shgo(objective, bounds, n=1000, iters=5)

        if result.fun == 0:
            # If a valid solution is found (objective function is zero)
            return result.x.reshape(1, -1)
        else:
            raise ValueError("Could not find an interest sample within the specified region.")

    def set_outlier_regions_centers(self, n_outlier_regions: int):

        # First outlier region at the origin
        X_outliers = np.zeros((1, self.n_dim_X))

        # Add more outlier regions if needed
        if n_outlier_regions > 1:
            additional_outliers = np.random.rand(n_outlier_regions - 1, n_dim_X)
            X_outliers = np.vstack([X_outliers, additional_outliers])

        return X_outliers

    def remove_close_outliers(self) -> np.ndarray:
        """
        Remove outliers that are too close to the X_interest sample.
        """
        # Calculate vectors from X_interest to outliers
        vectors_to_outliers = self.X_outliers - self.X_interest

        # Calculate distances from X_interest to outliers
        distances = np.linalg.norm(vectors_to_outliers, axis=1)

        # Create a mask for outliers that are too close
        mask_too_close = distances < self.outlier_radius

        # Return only the outliers that are not too close
        return self.X_outliers[~mask_too_close]

In [9]:
# Set input space
n_dim_X = 2
X_bounds = np.array([(0, 1)]*n_dim_X)

# Define interest region
interest_region = np.array([[0.7, 0.8]])

# Set fast simulator
fast_simulator = FastSimulator(
    proxy_function=spherical_shell_interest_function,
    n_dim_X=n_dim_X,
    interest_region=interest_region,
    n_outlier_regions= 10,
    outlier_radius=0.05
)

# Set interest sample
y_interest = fast_simulator.proxy_function(fast_simulator.X_interest)
data_point = (fast_simulator.X_interest, y_interest.ravel(), 'SHGO found interest sample')

# Set outlier regions
X_outliers = fast_simulator.X_outliers
outliers_data = (X_outliers, np.zeros(X_outliers.shape[0]), f'Outlier regions of radius {fast_simulator.outlier_radius}')

# Plot heatmap
plot_heatmap(spherical_shell_interest_function, 'Spherical Shell Interest', X_bounds, interest_region, data_point, outliers_data)

FastSimulator - > Searching for interest sample...


In [10]:
# Set input space
n_dim_X = 2
X_bounds = np.array([(0, 1)]*n_dim_X)

# Define interest region
interest_region = np.array([[0.7, 0.8]])

# Set fast simulator
fast_simulator = FastSimulator(
    proxy_function=shubert_like_interest_function,
    n_dim_X=n_dim_X,
    interest_region=interest_region,
    n_outlier_regions= 10,
    outlier_radius=0.05
)

# Set interest sample
y_interest = fast_simulator.proxy_function(fast_simulator.X_interest)
data_point = (fast_simulator.X_interest, y_interest.ravel(), 'SHGO found interest sample')

# Set outlier regions
X_outliers = fast_simulator.X_outliers
outliers_data = (X_outliers, np.zeros(X_outliers.shape[0]), f'Outlier regions of radius {fast_simulator.outlier_radius}')

# Plot heatmap
plot_heatmap(shubert_like_interest_function, 'Shubert-like Interest', X_bounds, interest_region, data_point, outliers_data)

FastSimulator - > Searching for interest sample...
